<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [3]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])
print("All libraries have been imported.")

All libraries have been imported.


In [5]:
import folium
import pandas as pd
print("imported")

imported


In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
print("imported")

imported


If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [7]:
## Task 1: Mark all launch sites on a map
# Sample data: List of launch sites with their coordinates
launch_sites = pd.DataFrame([
    {"name": "Kennedy Space Center", "lat": 28.5721, "lon": -80.6480},
    {"name": "Cape Canaveral Space Force Station", "lat": 28.3922, "lon": -80.6077},
    {"name": "Vandenberg Space Force Base", "lat": 34.6328, "lon": -120.6108},
    {"name": "Baikonur Cosmodrome", "lat": 45.9647, "lon": 63.3050},
    {"name": "Tanegashima Space Center", "lat": 30.4000, "lon": 130.9660}
])

# Initialize map centered around the first launch site
map_center = [launch_sites["lat"].mean(), launch_sites["lon"].mean()]
m = folium.Map(location=map_center, zoom_start=3)

# Add marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Add markers to the map
for _, site in launch_sites.iterrows():
    folium.Marker(
        location=[site["lat"], site["lon"]],
        popup=site["name"],
        tooltip=site["name"],
        icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
    ).add_to(marker_cluster)

# Display map
m

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [12]:
import requests
import pandas as pd

# Define the URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"

# Fetch the content
response = requests.get(url)
response.raise_for_status()  # Check for errors

# Read into pandas
spacex_df = pd.read_csv(io.StringIO(response.text))

# Display the first few rows
spacex_df.head()



,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [9]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [21]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Display the map
site_map


We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [14]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [22]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [24]:
# Task 2: Mark the success/failed launches for each site on the map
import folium
import pandas as pd

# Load the dataset
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

# Initialize a map centered at an average location
map_center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# Define color function for markers (green for success, red for failure)
def get_marker_color(outcome):
    return "green" if outcome == "Success" else "red"

# Add markers for each launch
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row["Lat"], row["Long"]],
        popup=f"Launch Site: {row['Launch Site']}\nOutcome: {row['class']}",
        tooltip=row["Launch Site"],
        icon=folium.Icon(color=get_marker_color(row["class"]))
    ).add_to(site_map)

# Display the map
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [25]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [26]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [27]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
import folium
from folium.plugins import MarkerCluster

# Initialize the map centered at a location (e.g., NASA Johnson Space Center)
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Function to determine the marker color based on launch success/failure (class value)
def get_marker_color(class_value):
    if class_value == 1:
        return 'green'  # Success: Green marker
    elif class_value == 0:
        return 'red'  # Failure: Red marker
    else:
        return 'gray'  # Default color for any other class values (if applicable)

# Apply the function to create a new 'marker_color' column in spacex_df
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

# Display the updated DataFrame (optional for verification)
print(spacex_df[['class', 'marker_color']].head())


   class marker_color
0      0          red
1      0          red
2      0          red
3      0          red
4      0          red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [32]:
# Add marker_cluster to current site_map
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
# Create the MarkerCluster object again (if not already created)
# Create the MarkerCluster object again (if not already created)
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each row in spacex_df to create and add markers
for index, record in spacex_df.iterrows():
    # Check if 'marker_color' column exists, if not create it
    if 'marker_color' not in record:
        # Apply a function to check the value of class column
        # If class=1, marker_color value will be green
        # If class=0, marker_color value will be red
        record['marker_color'] = 'green' if record['class'] == 1 else 'red'

    # Create a Marker with the location and popup info
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # Latitude and Longitude of the launch site
        popup=folium.Popup(f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}", max_width=200),
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='info-sign', prefix='fa')  # Icon color based on success/failure
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add the marker_cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [43]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [33]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [34]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [38]:
print(spacex_df.columns)

Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long', 'marker_color'],
      dtype='object')


In [42]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Coordinates of the closet COASTLINE


# Coordinates of the CCAFS LC-40 Launch Site
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Create the base map centered on the Launch Site
launch_site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Function to calculate the distance between two coordinates using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371.0
    
    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Coordinates of the closest COASTLINE
coastline_lat = 28.56321
coastline_lon = -80.56801

# Calculate distance to the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f'Distance to Coastline: {round(distance_coastline, 2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(100, -10),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:200px;">Coastline {round(distance_coastline, 2)} km</div>',
    )
).add_to(launch_site_map)

# Display the map
launch_site_map




Distance to Coastline: 0.92 km


In [44]:
# Coordinates of the closet RAIL LINE
rail_line_lat = 28.57209
rail_line_lon = -80.58520
distance_rail_line = calculate_distance(launch_site_lat, launch_site_lon, rail_line_lat, rail_line_lon)
print(f'Distance to Rail Line: {round(distance_rail_line,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [rail_line_lat, rail_line_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(-40,-20),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:200px;">Rail Line {round(distance_rail_line,2)} km</div>',
        )
    ).add_to(launch_site_map)

Distance to Rail Line: 1.33 km


In [45]:
# Coordinates of a point on the ROAD circling the launch site
road_lat = 28.561695
road_lon = -80.575553
distance_road = calculate_distance(launch_site_lat, launch_site_lon, road_lat, road_lon)
print(f'Distance to Road: {round(distance_road,2)} km')

# Create and add a folium.Marker on the selected point on the map
distance_marker = folium.Marker(
    [road_lat, road_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(10,0),
        html=f'<div style="font-size:12px; color:#222; font-weight:bold; width:160px;">Perimeter Road {round(distance_road,2)} km</div>',
        )
    ).add_to(launch_site_map)

Distance to Road: 0.19 km


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [49]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

# Coordinates of the CCAFS LC-40 Launch Site
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordinates of the closest COASTLINE
coastline_lat = 28.56367
coastline_lon = -80.57163

# Function to calculate the distance between two points (latitude, longitude)
def calculate_distance(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, sqrt, atan2

    # Radius of Earth in km
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula to calculate distance
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

# Calculate the distance to the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create and add a folium.Marker on the closest coastline point with the distance as the popup text
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
).add_to(site_map)

# Create a PolyLine object connecting the launch site and the coastline
coordinates = [(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [51]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Coordinates of the CCAFS LC-40 Launch Site
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordinates of the nearest city (for example, Cocoa Beach, FL)
city_lat = 28.3852
city_lon = -80.6077

# Function to calculate the distance between two points (latitude, longitude)
def calculate_distance(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, sqrt, atan2

    # Radius of Earth in km
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula to calculate distance
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance

# Calculate the distance to the nearest city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create and add a folium.Marker on the city location with the distance as the popup text
distance_marker = folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_city),
    )
).add_to(site_map)

# Create a PolyLine object connecting the launch site and the city
coordinates = [(launch_site_lat, launch_site_lon), (city_lat, city_lon)]
lines = folium.PolyLine(locations=coordinates, weight=2, color='green')

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
